<a href="https://colab.research.google.com/github/Arohrba5/Synthetic-Control-Causal-Inference-with-Covid-Data/blob/main/MITx_IDS_S24x_SyntheticInterventionRecitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt